Hausaufgabe5_Measurament Problems

#Datensatzgeschichte
In diesem Projekt werden wir Bewertungsdaten von einer E-Commerce-Plattform verwenden.
Der Datensatz enthält Benutzerbewertungen und Kommentare zu einem Produkt.
Die im Datensatz enthaltenen Variablen sind wie folgt:
reviewerID: Bewerter (Benutzer) ID
productID: Produkt ID
reviewerName: Benutzer Name
helpful: Nützlichkeitsbewertungsgrad
reviewText: Bewertungstext
overall: Produktbewertung (1-5)
summary: Bewertungsübersicht
unixReviewTime: Bewertungszeit (Unix-Zeitstempel)
reviewTime: Bewertungszeit (im rohen Format)
day_diff: Anzahl der Tage seit der Bewertung
helpfull_yes: Anzahl der als nützlich bewerteten Stimmen
total vote: Gesamtanzahl der für die Bewertung abgegebenen Stimmen



In [19]:
import pandas as pd
import seaborn as sns
import math
import scipy.stats as st

In [20]:
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.float_format", lambda x: "%5f" % x)

In [21]:
df= pd.read_csv(r"C:\Users\User Laptop\Desktop\phyton\phyton_ders\ödevler\ödev5\amazon_review.csv")


In [23]:
df= df.rename(columns ={"asin": "productID"})

In [24]:
df.head(8)

,reviewerID,productID,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.000000,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.000000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.000000,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.000000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.000000,best deal around,1373673600,2013-07-13,513,0,0
5,A2IDCSC6NVONIZ,B007WTAJTO,2Cents!,"[0, 0]",It's mini storage. It doesn't do anything els...,5.000000,Not a lot to really be said,1367193600,2013-04-29,588,0,0
6,A26YHXZD5UFPVQ,B007WTAJTO,2K1Toaster,"[0, 0]",I have it in my phone and it never skips a bea...,5.000000,Works well,1382140800,2013-10-19,415,0,0
7,A3CW0ZLUO5X2B1,B007WTAJTO,"35-year Technology Consumer ""8-tracks to 802.11""","[0, 0]",It's hard to believe how affordable digital ha...,5.000000,32 GB for less than two sawbucks...what's not ...,1404950400,2014-10-07,62,0,0


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      4915 non-null   object 
 1   productID       4915 non-null   object 
 2   reviewerName    4914 non-null   object 
 3   helpful         4915 non-null   object 
 4   reviewText      4914 non-null   object 
 5   overall         4915 non-null   float64
 6   summary         4915 non-null   object 
 7   unixReviewTime  4915 non-null   int64  
 8   reviewTime      4915 non-null   object 
 9   day_diff        4915 non-null   int64  
 10  helpful_yes     4915 non-null   int64  
 11  total_vote      4915 non-null   int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 460.9+ KB


#Aufgabe_1 Berechnen Sie die durchschnittliche Bewertung basierend auf aktuellen Bewertungen

In dieser Aufgabe werden Sie die durchschnittliche Bewertung eines Produkts anhand aktueller Bewertungen gewichtet berechnen 
(gewichteter Durchschnitt) und mit der bestehenden Durchschnittsbewertung vergleichen
1-Berechnen Sie die aktuelle durchschnittliche Bewertung des Produkts.
2-Berechnen Sie den gewichteten Durchschnitt basierend auf dem Datum und geben Sie neueren Daten mehr Gewicht
3-Vergleichen Sie die gewichtete durchschnittliche Bewertung mit der aktuellen durchschnittlichen Bewertung und kommentieren Sie sie


1-Berechnen Sie die aktuelle durchschnittliche Bewertung des Produkts.

In [28]:
df.groupby("productID")["overall"].mean()

productID
B007WTAJTO   4.587589
Name: overall, dtype: float64

2- Berechnen Sie den gewichteten Durchschnitt basierend auf dem Datum und geben Sie neueren Daten mehr Gewicht

In [30]:
df["reviewTime"]= pd.to_datetime(df["reviewTime"], errors= "coerce")  ##den Datentyp 'object' in 'datetime' umgewandelt

In [39]:
print(df["reviewTime"].dtype)

datetime64[ns]


In [43]:
current_date= df["reviewTime"].max()

In [44]:
df["days"]= (current_date - df["reviewTime"]).dt.days

In [45]:
df["days"].describe().T

count   4915.000000
mean     436.367040
std      209.439871
min        0.000000
25%      280.000000
50%      430.000000
75%      600.000000
max     1063.000000
Name: days, dtype: float64

In [48]:
def time_based_weighted_average(dataframe, w1=22, w2=24, w3=26, w4=28):
    return dataframe.loc[(dataframe["days"] <= 280), "overall"].mean() * w1 / 100 + \
           dataframe.loc[(dataframe["days"] > 280) & (dataframe["days"] <= 430), "overall"].mean() * w2 / 100 + \
           dataframe.loc[(dataframe["days"] > 430) & (dataframe["days"] <= 600), "overall"].mean() * w3 / 100 + \
           dataframe.loc[(dataframe["days"] > 600),"overall"].mean() * w4 / 100


time_based_weighted_average(df)

4.57933124861244

##"day_diff" definiert Anzahl der Tage seit der Bewertung, daher können Funktionen direkt darauf entwickelt werden, ohne Zeitumwandlungen durchzuführen

In [42]:
df["day_diff"].describe().T

count   4915.000000
mean     437.367040
std      209.439871
min        1.000000
25%      281.000000
50%      431.000000
75%      601.000000
max     1064.000000
Name: day_diff, dtype: float64

In [49]:
def time_based_weighted_average2(dataframe, w1=22, w2=24, w3=26, w4=28):
    return dataframe.loc[(dataframe["day_diff"] <= 281), "overall"].mean() * w1 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > 281) & (dataframe["day_diff"] <= 431), "overall"].mean() * w2 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > 431) & (dataframe["day_diff"] <= 601), "overall"].mean() * w3 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > 601),"overall"].mean() * w4 / 100


time_based_weighted_average2(df)

4.57933124861244

3-Vergleichen Sie die gewichtete durchschnittliche Bewertung mit der aktuellen durchschnittlichen Bewertung und kommentieren Sie sie

In [ ]:
#aktuellen durchschnittlichen Bewertung= 4.587589
#time_based_weighted_average= 4.57933124861244

#Aufgabe_2 Bestimmen Sie die 20 Kommentare, die auf der Produktdetailseite angezeigt werden sollen
1- Erstellen Sie die Variable 'helpful_no'
2- Berechnen Sie "score_pos_neg_diff", "score_average_rating", "Wilson Lower Bound"
3- Wählen Sie die 20 Bewertungen mit der höchsten Punktzahl basierend auf der Wilson-Lower-Bound-Methode aus

In [50]:
df["helpful_no"]= df["total_vote"] - df["helpful_yes"]

In [52]:
df.head()

,reviewerID,productID,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days,helpful_no
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.000000,Four Stars,1406073600,2014-07-23,138,0,0,137,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.000000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,408,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.000000,nothing to really say....,1356220800,2012-12-23,715,0,0,714,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.000000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,381,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.000000,best deal around,1373673600,2013-07-13,513,0,0,512,0


score_pos_neg_diff: Der Unterschied zwischen der Anzahl der nützlichen Stimmen und der Anzahl der nicht nützlichen Stimmen

In [53]:
def score_pos_neg_diff(up, down):
    return up - down
df["score_pos_neg_diff"] = df.apply(lambda x: score_pos_neg_diff(x["helpful_yes"], x["helpful_no"]), axis=1)

score_average_rating: Das Verhältnis der Anzahl der als nützlich bewerteten Stimmen zur Gesamtanzahl der Stimmen

In [54]:
def score_average_rating(up, down):
    if up + down == 0:
        return 0
    return up / (up + down)
df["score_average_rating"] = df.apply(lambda x: score_average_rating(x["helpful_yes"], x["helpful_no"]), axis=1)

wilsonlowerbound

In [55]:
def wilson_lower_bound(up, down, confidence=0.95):
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)
df["wilson_lower_bound"] = df.apply(lambda x:
wilson_lower_bound(x["helpful_yes"], x["helpful_no"]), axis=1)

3- Wählen Sie die 20 Bewertungen mit der höchsten Punktzahl basierend auf der Wilson-Lower-Bound-Methode aus

In [56]:
df[["overall","helpful_yes","total_vote","days","helpful_no", "score_pos_neg_diff", "score_average_rating","wilson_lower_bound"]]\
.sort_values("wilson_lower_bound", ascending= False).head(20)

,overall,helpful_yes,total_vote,days,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,5.000000,1952,2020,701,68,1884,0.966337,0.957544
3449,5.000000,1428,1505,802,77,1351,0.948837,0.936519
4212,1.000000,1568,1694,578,126,1442,0.925620,0.912139
317,1.000000,422,495,1032,73,349,0.852525,0.818577
4672,5.000000,45,49,157,4,41,0.918367,0.808109
1835,5.000000,60,68,282,8,52,0.882353,0.784651
3981,5.000000,112,139,776,27,85,0.805755,0.732136
3807,3.000000,22,25,648,3,19,0.880000,0.700442
4306,5.000000,51,65,822,14,37,0.784615,0.670334
4596,1.000000,82,109,806,27,55,0.752294,0.663595
